In [1]:
from convokit import Corpus, download
import re
import numpy as np

from utils.data_processing import DataProcesser
from utils.data_processing import TextProcesser
from utils import corpus_utils as corp
import pandas as pd
import pprint as pp
from model.config  import *
from model.data import *

/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/.venv/lib/python3.11/site-packages/convokit/coordination/coordination.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# DataFrame Loading

In [2]:
final_filepath_kodis = os.path.join(fine_processed_dir, 'kodis','processed_kodis.csv')
raw_filepath_kodis = os.path.join(fine_raw_dir, 'kodis', 'KODIS-EN.csv')
corpus_kodis_filepath = os.path.join(corpus_dir, "custom")
corp.setConversationMetadata(["buyer_is_AI", "seller_is_AI", "convo_len", "provided_outcome"
    ,"s_SVI_instrumental", "s_SVI_self", "s_SVI_process", "s_SVI_relationship"])
test = DataProcesser(final_filepath_kodis)
kodis_corpus = corp.corpusBuilder(test)

Row Index in columns
*** 0 utterances with timestamp == 0 after imputation ***


,id,speaker,conversation_id,reply_to,timestamp,text
0,utt0_con0,Seller_0,utt0_con0,None,1699388149,Your sudden demand for a refund is unwarranted...
1,utt1_con0,Buyer_0,utt0_con0,utt0_con0,1699388150,"This was not the item I ordered, please provid..."
2,utt2_con0,Seller_0,utt0_con0,utt1_con0,1699388170,How about a compromise? I can offer you a part...
3,utt3_con0,Buyer_0,utt0_con0,utt2_con0,1699388217,"No, this was not the item I ordered. I have a ..."
4,utt4_con0,Seller_0,utt0_con0,utt3_con0,1699388235,"I can scrutinize your claim, but this requires..."
...,...,...,...,...,...,...
39463,utt5_con2850,Buyer_2850,utt0_con2850,utt4_con2850,1718879890,Thank you I appreciate that. I shall remove m...
39464,utt6_con2850,Seller_2850,utt0_con2850,utt5_con2850,1718879909,"Deal. Once I see changes on your end, I'll ini..."
39465,utt7_con2850,Buyer_2850,utt0_con2850,utt6_con2850,1718879998,I understand - please do follow through. I ha...
39466,utt8_con2850,Seller_2850,utt0_con2850,utt7_con2850,1718880018,"You have my word, I will follow through. A sna..."


,id
0,Seller_0
1,Buyer_0
2,Seller_0
3,Buyer_0
4,Seller_0
...,...
39463,Buyer_2850
39464,Seller_2850
39465,Buyer_2850
39466,Seller_2850


,id,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
0,utt0_con0,False,True,16,1,NaN,NaN,NaN,NaN
1,utt0_con1,False,False,14,0,7.0,5.5,7.0,6.5
2,utt0_con2,False,True,12,0,NaN,NaN,NaN,NaN
3,utt0_con3,True,False,12,0,7.0,7.0,7.0,5.0
4,utt0_con4,True,False,14,0,4.5,5.0,4.5,5.5
...,...,...,...,...,...,...,...,...,...
2846,utt0_con2846,False,True,16,0,NaN,NaN,NaN,NaN
2847,utt0_con2847,False,True,10,0,NaN,NaN,NaN,NaN
2848,utt0_con2848,False,True,15,0,NaN,NaN,NaN,NaN
2849,utt0_con2849,False,False,11,0,5.0,6.0,6.5,5.0


39468it [00:01, 34264.08it/s]


Utterance conversation_ids missing from convos.id: set()
Conversation ids in convos not used by any utterance: set()
Every utterance.conversation_id exists in convos.id? True
Every convos.id is referred to by at least one utterance? True
2851 unique IDs in your utterance DF
['utt0_con0' 'utt0_con1' 'utt0_con2' 'utt0_con3' 'utt0_con4' 'utt0_con5'
 'utt0_con6' 'utt0_con7' 'utt0_con8' 'utt0_con9']
Unique conversation_id’s in your utterance DF: 2851


# Testing Filtering

In [3]:
test.filterMatches("Accept Deal")
test.filterMatches("I Walk Away")
test.filterMatches("Submit agreement")
no_accept_df= test.filterRows("message", exclude_val="Accept Deal", case_ex=True)
no_submit_df= test.filterRows("message", exclude_val="I Walk Away", case_ex=True)
sub_filtered = test.filterRows("message", exclude_val="Submitted agreement", case_ex=True)
no_last_df = test.filterRows("message", exclude_val=["Accept Deal", "I Walk Away"], case_in=True)
display(sub_filtered)
test_corpus = corp.corpusBuilder(test)

,timestamp,message,uttidx,speaker_id,is_AI,row_idx,match_idx,Case Match Type,convo_len
0,nan,Your sudden demand for a refund is unwarranted...,0,Seller_0,True,0,False,NaN,16
1,1699388150,"This was not the item I ordered, please provid...",1,Buyer_0,False,0,False,NaN,16
2,1699388170,How about a compromise? I can offer you a part...,2,Seller_0,True,0,False,NaN,16
3,1699388217,"No, this was not the item I ordered. I have a ...",3,Buyer_0,False,0,False,NaN,16
4,1699388235,"I can scrutinize your claim, but this requires...",4,Seller_0,True,0,False,NaN,16
...,...,...,...,...,...,...,...,...,...
39462,1718879809,"Alright, I can see this is really important fo...",4,Seller_2850,True,2850,False,NaN,10
39463,1718879890,Thank you I appreciate that. I shall remove m...,5,Buyer_2850,False,2850,False,NaN,10
39464,1718879909,"Deal. Once I see changes on your end, I'll ini...",6,Seller_2850,True,2850,False,NaN,10
39465,1718879998,I understand - please do follow through. I ha...,7,Buyer_2850,False,2850,False,NaN,10


*** 0 utterances with timestamp == 0 after imputation ***


,id,speaker,conversation_id,reply_to,timestamp,text
0,utt0_con0,Seller_0,utt0_con0,None,1699388149,Your sudden demand for a refund is unwarranted...
1,utt1_con0,Buyer_0,utt0_con0,utt0_con0,1699388150,"This was not the item I ordered, please provid..."
2,utt2_con0,Seller_0,utt0_con0,utt1_con0,1699388170,How about a compromise? I can offer you a part...
3,utt3_con0,Buyer_0,utt0_con0,utt2_con0,1699388217,"No, this was not the item I ordered. I have a ..."
4,utt4_con0,Seller_0,utt0_con0,utt3_con0,1699388235,"I can scrutinize your claim, but this requires..."
...,...,...,...,...,...,...
39463,utt5_con2850,Buyer_2850,utt0_con2850,utt4_con2850,1718879890,Thank you I appreciate that. I shall remove m...
39464,utt6_con2850,Seller_2850,utt0_con2850,utt5_con2850,1718879909,"Deal. Once I see changes on your end, I'll ini..."
39465,utt7_con2850,Buyer_2850,utt0_con2850,utt6_con2850,1718879998,I understand - please do follow through. I ha...
39466,utt8_con2850,Seller_2850,utt0_con2850,utt7_con2850,1718880018,"You have my word, I will follow through. A sna..."


,id
0,Seller_0
1,Buyer_0
2,Seller_0
3,Buyer_0
4,Seller_0
...,...
39463,Buyer_2850
39464,Seller_2850
39465,Buyer_2850
39466,Seller_2850


,id,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
0,utt0_con0,False,True,16,1,NaN,NaN,NaN,NaN
1,utt0_con1,False,False,14,0,7.0,5.5,7.0,6.5
2,utt0_con2,False,True,12,0,NaN,NaN,NaN,NaN
3,utt0_con3,True,False,12,0,7.0,7.0,7.0,5.0
4,utt0_con4,True,False,14,0,4.5,5.0,4.5,5.5
...,...,...,...,...,...,...,...,...,...
2846,utt0_con2846,False,True,16,0,NaN,NaN,NaN,NaN
2847,utt0_con2847,False,True,10,0,NaN,NaN,NaN,NaN
2848,utt0_con2848,False,True,15,0,NaN,NaN,NaN,NaN
2849,utt0_con2849,False,False,11,0,5.0,6.0,6.5,5.0


39468it [00:01, 35785.44it/s]


Utterance conversation_ids missing from convos.id: set()
Conversation ids in convos not used by any utterance: set()
Every utterance.conversation_id exists in convos.id? True
Every convos.id is referred to by at least one utterance? True
2851 unique IDs in your utterance DF
['utt0_con0' 'utt0_con1' 'utt0_con2' 'utt0_con3' 'utt0_con4' 'utt0_con5'
 'utt0_con6' 'utt0_con7' 'utt0_con8' 'utt0_con9']
Unique conversation_id’s in your utterance DF: 2851


# Testing Context Creation

In [4]:


def preprocess_utterances_test(utt_df, voc):
    utt_df = utt_df.copy()
    utt_df["tokens"] = (utt_df["text"].map(tokenize) #-> tokens for eack text
        .map(lambda toks: [t if t in voc.word2index else "UNK" for t in toks])
    )
    if utt_label_metadata is None:
        utt_df["is_attack"] = 0
    else:
        utt_df["is_attack"] = (utt_df[utt_label_metadata].fillna(0).astype(int))
    return utt_df

def load_pairs_test(voc,utt_df, conv_df, last_only: bool = False):
    """
    Returns (train_pairs, val_pairs, test_pairs), where each is a list of
    (context: List[List[str]], reply: List[str], label: int, comment_id: str).
    """
    utts = preprocess_utterances_test(utt_df, voc)

    # Prepare split IDs
    splits = {}
    for split in ("train", "val", "test"):
        ids = conv_df.index[conv_df["split"] == split].unique()
        splits[split] = set(ids)


    def make_pairs_for_split_test(ids_set):
        pairs = []
        for convo_id, dialog_df in utts[utts["conversation_id"].isin(ids_set)] \
                                   .groupby("conversation_id", sort=False):
           
            dialog = dialog_df[["tokens", "is_attack", "conversation_id"]].to_dict("records")
            if utt_label_metadata is None and dialog:
                last_id = dialog[-1]["conversation_id"]
                conv_label = conv_df.at[convo_id, label_metadata]
                dialog.append({
                    "tokens": ["UNK"],
                    "is_attack": conv_label,
                    "id": f"{last_id}_dummyreply"
                })
            idxs = [len(dialog) - 1] if last_only else range(1, len(dialog))
            for idx in idxs:
                context = [u["tokens"][: MAX_LENGTH - 1] for u in dialog[max(0, idx - CONTEXT_SIZE) : idx]]
                reply     = dialog[idx]["tokens"][: MAX_LENGTH - 1]
                label     = dialog[idx]["is_attack"]
                comment_id = dialog[idx - 1]["conversation_id"]
                pairs.append((context, reply, label, comment_id))
        return pairs
    train_pairs = make_pairs_for_split_test(splits["train"])
    val_pairs   = make_pairs_for_split_test(splits["val"])
    test_pairs  = make_pairs_for_split_test(splits["test"])
    return train_pairs, val_pairs, test_pairs

In [5]:
voc = loadPrecomputedVoc(corpus_name, word2index_path, index2word_path)


In [6]:
utts = kodis_corpus.get_utterances_dataframe()
display(utts)

,timestamp,text,speaker,reply_to,conversation_id,vectors
id,,,,,,
utt0_con0,1699388149,Your sudden demand for a refund is unwarranted...,Seller_0,None,utt0_con0,[]
utt1_con0,1699388150,"This was not the item I ordered, please provid...",Buyer_0,utt0_con0,utt0_con0,[]
utt2_con0,1699388170,How about a compromise? I can offer you a part...,Seller_0,utt1_con0,utt0_con0,[]
utt3_con0,1699388217,"No, this was not the item I ordered. I have a ...",Buyer_0,utt2_con0,utt0_con0,[]
utt4_con0,1699388235,"I can scrutinize your claim, but this requires...",Seller_0,utt3_con0,utt0_con0,[]
...,...,...,...,...,...,...
utt5_con2850,1718879890,Thank you I appreciate that. I shall remove m...,Buyer_2850,utt4_con2850,utt0_con2850,[]
utt6_con2850,1718879909,"Deal. Once I see changes on your end, I'll ini...",Seller_2850,utt5_con2850,utt0_con2850,[]
utt7_con2850,1718879998,I understand - please do follow through. I ha...,Buyer_2850,utt6_con2850,utt0_con2850,[]


In [15]:
conv_df =kodis_corpus.get_conversations_dataframe()
display(conv_df)


,vectors,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
id,,,,,,,,,
utt0_con0,[],False,True,16,1,NaN,NaN,NaN,NaN
utt0_con1,[],False,False,14,0,7.0,5.5,7.0,6.5
utt0_con2,[],False,True,12,0,NaN,NaN,NaN,NaN
utt0_con3,[],True,False,12,0,7.0,7.0,7.0,5.0
utt0_con4,[],True,False,14,0,4.5,5.0,4.5,5.5
...,...,...,...,...,...,...,...,...,...
utt0_con2846,[],False,True,16,0,NaN,NaN,NaN,NaN
utt0_con2847,[],False,True,10,0,NaN,NaN,NaN,NaN
utt0_con2848,[],False,True,15,0,NaN,NaN,NaN,NaN


In [ ]:
utts = preprocess_utterances_test(utts, voc)

for convo_id, dialog_df in utts.groupby("conversation_id", sort=False):
    dialog = dialog_df[["tokens", "is_attack", "conversation_id"]].to_dict("records")   
    if utt_label_metadata is None and dialog:
            last_id = dialog[-1]["conversation_id"]
            conv_label = conv_df.at[convo_id, f"meta.{label_metadata}"]
            dialog.append({
                "tokens": ["UNK"],
                "is_attack": conv_label,
                "utt_id": f"{last_id}_dummyreply"
            })
            print(dialog)



KeyError: 'provided_outcome'